In [1]:
import scipy.interpolate as interp
from ruspy.estimation.estimation_transitions import create_transition_matrix
from ruspy.model_code.cost_functions import calc_obs_costs
from ruspy.model_code.cost_functions import lin_cost
from ruspy.model_code.fix_point_alg import calc_fixp
from config import DATA_DIR

In [2]:
# Load data. This step will be replaced when we have different data.
df = performance_dataframe()
df_meaned = df.mean(level=0)

# Evaluation grid

Now creat an evaluation grid on which we will later interpolate the performance and then apply the decision criteria.

In [3]:
# Load the probability grid from the data.
grid_probs = pkl.load(open(f"{DATA_DIR}/grid.pkl", "rb"))

In [5]:
# The omega grid only to the maximum selected omega.
omega_grid = np.arange(0, 1, 0.01)
one_dim_prob_grid = np.arange(0, 0.9, 0.01)
one_dim_prob_grid[0] += 0.005
interpol_prob_grid = create_prob_grid(one_dim_prob_grid, 3)
adapted_grid = interpol_prob_grid[(interpol_prob_grid[:, 0] >= 0.05) & (interpol_prob_grid[:, 1] >= 0.05) & (interpol_prob_grid[:, 0] <= 0.9) & (interpol_prob_grid[:, 1] <= 0.9) & (interpol_prob_grid[:, 2] > 0.1)]

# Interpolation and apply criteria

In [71]:
omega_interpol_vals = np.zeros((grid_probs.shape[0], len(omega_grid)))
for id_grid, gridpoint in enumerate(grid_probs):
    omega_interpol_vals[id_grid, :] = interp.griddata(VAL_STRATS, omega_vals[id_grid, :], omega_grid, method="linear")

In [8]:
omega_interpol_vals.shape

(55, 100)

In [9]:
final_interpol_vals = np.zeros((adapted_grid.shape[0], len(omega_grid)))
for omega_id in range(len(omega_grid)):
    final_interpol_vals[:, omega_id] = interp.griddata(grid_probs[:, :2], omega_interpol_vals[:, omega_id], adapted_grid[:, :2], method="linear")

## Minimax regret

In [ ]:
# omegas_mean = np.mean(omega_interpol_vals, axis=0)
# normalized_mean = filter_normalize(omegas_mean)


In [80]:
# regrets_omega = df_meaned.loc[:, "best"].to_numpy().reshape(55, 1) - omega_interpol_vals
# max_regret = np.max(regrets_omega, axis=0)
# normalized_regret = filter_normalize(max_regret * -1)